## Comparación Técnicas de Codificación de Variables Categóricas

En esta lección, vamos a comparar el desempeño de de las diferentes técnicas para codificar variables categóricas:

Vamos a comparar:

- Codificación One-hot 
- Reemplazar etiquetas por el conteo
- Ordenar etiquetas de acuerdo al target
- Codificación por la media
- WoE

y vamos a usar los datos del Titanic.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [2]:
# carguemos el titanic dataset

# solo vamos a usar estar columnas en el demo
cols = ['pclass', 'age', 'sibsp', 'parch', 'fare',
        'sex', 'cabin', 'embarked', 'survived']

data = pd.read_csv('../titanic.csv', usecols=cols)

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [3]:
# revisemos por datos faltantes

data.isnull().sum()

pclass         0
survived       0
sex            0
age          263
sibsp          0
parch          0
fare           1
cabin       1014
embarked       2
dtype: int64

In [4]:
# removamos las observaciones con datos ausentes NA en fare & embarked
data.dropna(subset=['fare', 'embarked'], inplace=True)

In [5]:
# Ahora extraemos la primera letra de la variable cabin
# para facilitar el demo

data['cabin'] = data['cabin'].astype(str).str[0]

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B,S
1,1,1,male,0.9167,1,2,151.5500,C,S
2,1,0,female,2.0000,1,2,151.5500,C,S
3,1,0,male,30.0000,1,2,151.5500,C,S
4,1,0,female,25.0000,1,2,151.5500,C,S


In [6]:
# remover las observaciones donde cabin = T
# ya que solo son unas pocas

data = data[data['cabin'] != 'T']

In [7]:
# separemos en segmentos de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels='survived', axis=1),  # predictores
    data['survived'],  # target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((913, 8), (392, 8))

In [8]:
# Reemplacemos los valores nulos por la media en las 
# variables numéricas por la media


def impute_na(df, variable, value):
    df[variable].fillna(value, inplace=True)


impute_na(X_test, 'age', X_train['age'].mean())
impute_na(X_train, 'age',  X_train['age'].mean())
# ahora vamos a imputar primero en el segmento de entrenamiento, de esta forma el
# valor de la mediana puede ser usado en ambos segmentos el de entrenamiento y prueba

In [9]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,female,30.00000,1,0,13.8583,n,C
698,3,male,18.00000,0,0,8.6625,n,S
1291,3,male,29.79847,0,0,8.7125,n,S
1229,3,male,27.00000,0,0,8.6625,n,S
118,1,male,29.79847,0,0,26.5500,D,S


In [10]:
# revisemos que no tenemos datos ausentes NA
# despues de la imputación
X_train.isnull().sum(), X_test.isnull().sum()

(pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64, pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64)

### Codificación One-Hot 

In [11]:
def get_OHE(df):

    df_OHE = pd.concat(
        [df[['pclass', 'age', 'sibsp', 'parch', 'fare']],
         pd.get_dummies(df[['sex', 'cabin', 'embarked']], drop_first=True)],
        axis=1)

    return df_OHE


X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

X_train_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
402,2,30.00000,1,0,13.8583,0,0,0,0,0,0,0,1,0,0
698,3,18.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
1291,3,29.79847,0,0,8.7125,1,0,0,0,0,0,0,1,0,1
1229,3,27.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
118,1,29.79847,0,0,26.5500,1,0,0,1,0,0,0,0,0,1


In [12]:
X_test_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
586,2,29.00000,1,0,26.0000,0,0,0,0,0,0,0,1,0,1
200,1,46.00000,0,0,75.2417,1,0,1,0,0,0,0,0,0,0
831,3,40.00000,1,6,46.9000,1,0,0,0,0,0,0,1,0,1
1149,3,29.79847,0,0,7.7208,0,0,0,0,0,0,0,1,1,0
393,2,25.00000,0,0,31.5000,1,0,0,0,0,0,0,1,0,1


### Codificación por conteo de frecuencia

In [13]:
def categorical_to_counts(df_train, df_test):

    # copia temporal de los dataframes originales 
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy()

    for col in ['sex', 'cabin', 'embarked']:

        # dicccionario con el mapeo de categoria a conteo de frecuencia
        counts_map = df_train_temp[col].value_counts().to_dict()

        # reemplazar las etiquetas por el conteo respectivo
        df_train_temp[col] = df_train_temp[col].map(counts_map)
        df_test_temp[col] = df_test_temp[col].map(counts_map)

    return df_train_temp, df_test_temp


X_train_count, X_test_count = categorical_to_counts(X_train, X_test)

X_train_count.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,326,30.00000,1,0,13.8583,702,184
698,3,587,18.00000,0,0,8.6625,702,647
1291,3,587,29.79847,0,0,8.7125,702,647
1229,3,587,27.00000,0,0,8.6625,702,647
118,1,587,29.79847,0,0,26.5500,33,647


### Codificación por Números Enteros Ordenados 


In [14]:
def categories_to_ordered(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # ordenar las categorias de acuerdo al promedio del target
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().sort_values().index

        #  diccionario con el mapeo de categoria ordenadas a un número ordinal
        ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

        # reemplazar las etiquetas por el numero entero ordenado
        df_train_temp[col] = df_train[col].map(ordinal_label)
        df_test_temp[col] = df_test[col].map(ordinal_label)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ordered, X_test_ordered = categories_to_ordered(
    X_train, X_test, y_train, y_test)

X_train_ordered.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,1,30.00000,1,0,13.8583,0,2
698,3,0,18.00000,0,0,8.6625,0,0
1291,3,0,29.79847,0,0,8.7125,0,0
1229,3,0,27.00000,0,0,8.6625,0,0
118,1,0,29.79847,0,0,26.5500,5,0


### Codificación por la Media

In [23]:
def categories_to_mean(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # calcular el promedio del target por categoria
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().to_dict()

       # reemplazar las etiquetas por el promedio del target
        df_train_temp[col] = df_train[col].map(ordered_labels)
        df_test_temp[col] = df_test[col].map(ordered_labels)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_mean, X_test_mean = categories_to_mean(
    X_train, X_test, y_train, y_test)

X_train_mean.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.730061,30.00000,1,0,13.8583,0.292023,0.516304
698,3,0.173765,18.00000,0,0,8.6625,0.292023,0.332303
1291,3,0.173765,29.79847,0,0,8.7125,0.292023,0.332303
1229,3,0.173765,27.00000,0,0,8.6625,0.292023,0.332303
118,1,0.173765,29.79847,0,0,26.5500,0.696970,0.332303


### WoE

In [22]:
def categories_to_woe(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # crear df con las diferentes partes de la ecuación del WoE
        # prob survived =1
        prob_df = pd.DataFrame(df_train_temp.groupby([col])['survived'].mean())

        # prob survived = 0
        prob_df['died'] = 1-prob_df.survived

        # calcular WoE
        prob_df['WoE'] = np.log(prob_df.survived/prob_df.died)

        # capturar WoE en diccionario
        woe = prob_df['WoE'].to_dict()

        # # reemplazar las etiquetas por WoE
        df_train_temp[col] = df_train[col].map(woe)
        df_test_temp[col] = df_test[col].map(woe)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_woe, X_test_woe = categories_to_woe(X_train, X_test, y_train, y_test)

X_train_woe.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.994934,30.00000,1,0,13.8583,-0.885580,0.065241
698,3,-1.559176,18.00000,0,0,8.6625,-0.885580,-0.697788
1291,3,-1.559176,29.79847,0,0,8.7125,-0.885580,-0.697788
1229,3,-1.559176,27.00000,0,0,8.6625,-0.885580,-0.697788
118,1,-1.559176,29.79847,0,0,26.5500,0.832909,-0.697788


### Desempeño Random Forest

In [16]:
# crear una función para construir modelo Random Forest
# y comparar desempeño en los segmentos de entrenamiento y prueba


def run_randomForests(X_train, X_test, y_train, y_test):

    rf = RandomForestClassifier(n_estimators=50, random_state=39, max_depth=3)
    rf.fit(X_train, y_train)

    print('Segmento entrenamiento')
    pred = rf.predict_proba(X_train)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Segmento prueba')
    pred = rf.predict_proba(X_test)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [18]:
# OHE
run_randomForests(X_train_OHE, X_test_OHE, y_train, y_test)

Segmento entrenamiento
Random Forests roc-auc: 0.8488938507340109
Segmento prueba
Random Forests roc-auc: 0.8072730715135779


In [19]:
# Conteos
run_randomForests(X_train_count, X_test_count, y_train, y_test)

Segmento entrenamiento
Random Forests roc-auc: 0.8654552920644698
Segmento prueba
Random Forests roc-auc: 0.8194309206967434


In [20]:
# Etiquetas ordenadas
run_randomForests(X_train_ordered, X_test_ordered, y_train, y_test)

Segmento entrenamiento
Random Forests roc-auc: 0.8669027820552304
Segmento prueba
Random Forests roc-auc: 0.8219733852645245


In [24]:
# Codificación por la media
run_randomForests(X_train_mean, X_test_mean, y_train, y_test)

Segmento entrenamiento
Random Forests roc-auc: 0.867010573863053
Segmento prueba
Random Forests roc-auc: 0.8207562479714378


In [22]:
# WoE
run_randomForests(X_train_woe, X_test_woe, y_train, y_test)

Train set
Random Forests roc-auc: 0.867010573863053
Test set
Random Forests roc-auc: 0.8207562479714378


Comparando los valores de roc_auc values en los segmentos de prueba, podemos ver que la codificación One-Hot tiene el peor desempeño. Tiene sentido porque algoritmos basados en  árboles no tienen un buen desempeño con datos que tiene espacios de variables grandes.

El resto de la codificaciones tienen un desempeño similar. Lo cual tambien es de esperarse, ya que los algorimos basados en árboles son modelos no-linales, por lo tanto codificaciones basadas en la media del target no necesariamente mejorar su desempeño.

### Desempeño Regresión Logística

In [25]:
def run_logistic(X_train, X_test, y_train, y_test):

    # función para entrenar y evaluar desempeño de Regresión Logística
    logit = LogisticRegression(random_state=44, C=0.01)
    logit.fit(X_train, y_train)

    print('Segmento Entrenamiento')
    pred = logit.predict_proba(X_train)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Segmento Prueba')
    pred = logit.predict_proba(X_test)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [26]:
# OHE
run_logistic(X_train_OHE, X_test_OHE, y_train, y_test)

Segmento Entrenamiento
Logistic Regression roc-auc: 0.8253644389693051
Segmento Prueba
Logistic Regression roc-auc: 0.7973601644487721


/Users/natisangarita/python/courses/feml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
# conteos
run_logistic(X_train_count, X_test_count, y_train, y_test)

Segmento Entrenamiento
Logistic Regression roc-auc: 0.7814777743558157
Segmento Prueba
Logistic Regression roc-auc: 0.7226279346532511


/Users/natisangarita/python/courses/feml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
# Etiquetas ordenadas
run_logistic(X_train_ordered, X_test_ordered, y_train, y_test)

Segmento Entrenamiento
Logistic Regression roc-auc: 0.8233831228826608
Segmento Prueba
Logistic Regression roc-auc: 0.804581845721086


/Users/natisangarita/python/courses/feml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
# Codificación por la media
run_logistic(X_train_mean, X_test_mean, y_train, y_test)

Segmento Entrenamiento
Logistic Regression roc-auc: 0.7913792218458064
Segmento Prueba
Logistic Regression roc-auc: 0.762712322838905


/Users/natisangarita/python/courses/feml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [30]:
# WoE
run_logistic(X_train_woe, X_test_woe, y_train, y_test)

Segmento Entrenamiento
Logistic Regression roc-auc: 0.8450082127091674
Segmento Prueba
Logistic Regression roc-auc: 0.8166450286703452


/Users/natisangarita/python/courses/feml/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Para la Regresión Logística, el mejor desempeño se obtiene con la codificación One-Hot, ya que preserva la relación lineal entre las variables y el target y seguido por WoE y la codificación por números enteros ordenados.

Sin embargo, la codificación por conteo de frecuencia, tiene el peor desempeño ya que no crea una relación monotónica entre las variables y el target, y en este caso, la codificación promedia del target esta causando sobre-ajustes.